In [8]:
import os
import sys
sys.path.append('..')
import scrape_tournament_info

import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
def parse_division_name(tournament_url):
	tournament_id = scrape_tournament_info.get_id_from_url(tournament_url)
	# TODO: CHeck if tournament id in DB and to scrape is false
	for id, division_name in scrape_tournament_info.get_formats(tournament_url):
		parse_divisions(id, division_name, tournament_id)
	# TODO: Set tournament id in DB to scrape to true

parse_division_name("https://www.tabroom.com/index/tourn/postings/index.mhtml?tourn_id=29541")

id='272611', tournament_id='29541', division_name='CX Open', round='CXOP Round 1', url='https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1094093'
id='272610', tournament_id='29541', division_name='Congress Open', round='Session 2 CONOP', url='https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1094091'
id='272610', tournament_id='29541', division_name='Congress Open', round='Session 3 CONOP', url='https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1094352'
id='272610', tournament_id='29541', division_name='Congress Open', round='Session 1 CONOP', url='https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1094090'
id='272613', tournament_id='29541', division_name='DUO Open', round='Finals DUOOP', url='https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1128339'
id='272613', tournament_id='29541', division_name='DUO Open', round='Round 3 DUOOP

In [20]:
def parse_divisions(id, division_name, tournament_id):
	response = requests.post(
		"https://www.tabroom.com/index/tourn/postings/index.mhtml",
		data={
			'tourn_id': tournament_id,
			'event_id': id
		}
	)
	pattern = r"/index/tourn/postings/round\.mhtml\?tourn_id="+tournament_id+r"&round_id=\d+"
	urls = set('https://www.tabroom.com'+match for match in re.findall(pattern, response.text))
	for url in urls: parse_rounds(id, division_name, tournament_id, url)



In [19]:
def parse_rounds(id, division_name, tournament_id, url):
	response = requests.get(url)
	round = BeautifulSoup(response.text, 'html.parser')\
		.find_all('h4')[-1]\
		.get_text(strip=True)
	print(f"{id=}, {tournament_id=}, {division_name=}, {round=}, {url=}")
	# TODO: Add division to SQL

In [61]:


first_table = soup.find('table')

table_html = str(first_table)
pd.read_html(table_html)

/var/folders/6w/yqthf3cx1v1gtm_3th5g0nk00000gp/T/ipykernel_85283/2996684344.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(table_html)


[    Flt           Room                   Aff                  Neg  \
 0     1    Howarth 203          Northwood JS           Newport KC   
 1     1      Jones 206            Newport XT        Bellingham NP   
 2     2      Jones 206            Newport BL          Mount Si DR   
 3     1      Jones 208      Mercer Island JY  Project Dialogue WZ   
 4     2      Jones 208            Seattle JP            Sehome RS   
 5     1   McIntyre 202             Sehome EM         Riverside SH   
 6     2   McIntyre 202           Bellevue WL       Woodinville VP   
 7     1   McIntyre 203   Project Dialogue OX          Overlake YK   
 8     2   McIntyre 203           Puyallup KW           Redmond JW   
 9     1   McIntyre 204             Sehome AM         Interlake SY   
 10    2   McIntyre 204    Lake Washington AC        Bainbridge CM   
 11    1   McIntyre 320  Palisades Charter SM     Mercer Island AZ   
 12    2   McIntyre 320          Interlake JP          Overlake RM   
 13    1   Thompson 

In [58]:
first_table = soup.find('table')
print(first_table.prettify())

<table id="1094149">
 <thead>
  <tr class="yellowrow smallish padless">
   <th class="centeralign">
    Flt
   </th>
   <th class="centeralign">
    Room
   </th>
   <th class="centeralign">
    Aff
   </th>
   <th class="centeralign">
    Neg
   </th>
   <th class="centeralign smallish">
    Judge
   </th>
  </tr>
 </thead>
 <tbody>
  <tr class="smallish">
   <td class="centeralign">
    1
   </td>
   <td class="nospace">
    <span class="marno full">
     <span class="quarterspacer">
     </span>
     Howarth 203
    </span>
   </td>
   <td class="nospace padleft">
    <a class="white smallish padtop padbottom padleft" href="/index/tourn/postings/entry_record.mhtml?tourn_id=29541&amp;entry_id=5453086">
     Northwood JS
    </a>
   </td>
   <td class="nospace padleft">
    <a class="white smallish padtop padbottom padleft" href="/index/tourn/postings/entry_record.mhtml?tourn_id=29541&amp;entry_id=5474851">
     Newport KC
    </a>
   </td>
   <td class="smallish nospace padleft">
   

In [62]:


# Assuming 'link' contains the URL to be requested
response = requests.get(link)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first <table> element in the parsed HTML
first_table = soup.find('table')

if first_table:
    # Initialize lists to store table data
    table_data = []
    headers = []

    # Process table headers if they exist
    thead = first_table.find('thead')
    if thead:
        header_row = thead.find('tr')
        if header_row:
            headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

    # Process table rows
    tbody = first_table.find('tbody')
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            row_data = []
            for col in cols:
                # Get text content
                text = col.get_text(strip=True)
                # Find all links in the cell
                links = [a['href'] for a in col.find_all('a', href=True)]
                # Append text and links as a tuple
                row_data.append((text, links))
            table_data.append(row_data)

    # Convert to DataFrame
    if headers:
        df = pd.DataFrame(table_data, columns=headers)
    else:
        df = pd.DataFrame(table_data)

    # Print DataFrame
    print(df)
else:
    print("No table found.")


        Flt                 Room  \
0   (1, [])    (Howarth 203, [])   
1   (1, [])      (Jones 206, [])   
2   (2, [])      (Jones 206, [])   
3   (1, [])      (Jones 208, [])   
4   (2, [])      (Jones 208, [])   
5   (1, [])   (McIntyre 202, [])   
6   (2, [])   (McIntyre 202, [])   
7   (1, [])   (McIntyre 203, [])   
8   (2, [])   (McIntyre 203, [])   
9   (1, [])   (McIntyre 204, [])   
10  (2, [])   (McIntyre 204, [])   
11  (1, [])   (McIntyre 320, [])   
12  (2, [])   (McIntyre 320, [])   
13  (1, [])   (Thompson 374, [])   
14  (1, [])  (Wheelock 230A, [])   
15  (1, [])      (Wyatt 204, [])   
16  (2, [])      (Wyatt 204, [])   
17  (1, [])      (Wyatt 226, [])   
18  (2, [])      (Wyatt 226, [])   
19  (1, [])      (Wyatt 305, [])   
20  (2, [])      (Wyatt 305, [])   
21  (1, [])            (BYE, [])   

                                                  Aff  \
0   (Northwood JS, [/index/tourn/postings/entry_re...   
1   (Newport XT, [/index/tourn/postings/entry_reco...   


In [84]:
[df.loc[0][column][1] != [] for column in df.columns]

def has_link(column):
	return df.loc[0][column][1] != []
columns_with_links = list(filter(has_link, df.columns))
columns_with_links

['Aff', 'Neg', 'Judge']

In [89]:
team_or_judge_name, url = df[columns_with_links[0]][0]
team_or_judge_name, url

('Northwood JS',
 ['/index/tourn/postings/entry_record.mhtml?tourn_id=29541&entry_id=5453086'])

In [91]:
'https://www.tabroom.com'+url[0]

'https://www.tabroom.com/index/tourn/postings/entry_record.mhtml?tourn_id=29541&entry_id=5453086'

In [106]:
def get_name(url):
	response = requests.get('https://www.tabroom.com'+url[0])
	soup = BeautifulSoup(response.text, 'html.parser')
	first_h2 = soup.find_all('h4')
	elements = soup.find_all(class_="nospace semibold")

	# print (elements)
	if first_h2:
		return first_h2[-1]\
			.get_text(strip=True)\
			.replace("\n", "")\
			.replace("\t", "")\
			.split("&")
	else:
		return "No Title Found"

get_name(["/index/tourn/postings/entry_record.mhtml?tourn_id=29541&entry_id=5396865"])

['Kavya Rengan', 'Anna Charles']

In [107]:
get_name(url)

['Jasmine Shao']